# Práctica 1 - Clasificación de palabras

In [9]:
#Misc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import mean
from matplotlib.colors import ListedColormap
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
#CrossValidation
##K-folding
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
##GridSearch
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
#https://scikit-learn.org/stable/modules/model_evaluation.html
# Model
from sklearn.svm import SVC

## Preparación de los datos
En esta primera parte de la práctica vamos a preparar los datos para poder hacer la clasificación. Para ello haremos que cada muestra siga el siguiente formato: $C_1$ | $C_2$ | $C_3$ | .. | $C_n$ | $y$  (siendo $C$ una característica y $y$ el target).

Se va a codificar $y$ con los valores "0" para representar la clase _catala_ y "1" para la clase _angles_.

In [10]:
#Auxiliar function that reformats df to one that is more fitting to Classification Problems
def reformat(dataFrame):
    if dataFrame.columns[0]=="catala":
        dataFrame['y']=0
    else:
        dataFrame['y']=1
    
    dataFrame.rename(columns={dataFrame.columns[0]: 'word'}, inplace=True)
    return dataFrame

#RawData, it has been a little bit formatted before reading the csv, to facilitate the process.
raw=pd.read_csv("data/data.csv")
#Split df
catala, angles= raw.filter(['catala'], axis=1), raw.filter(['angles'], axis=1)
#Reformating
catala=reformat(catala)
angles=reformat(angles)

#Merging
wordsDF=pd.concat([catala,angles], axis=0)
#Shuffle the rows
wordsDF = wordsDF.sample(frac=1).reset_index(drop=True)



### Caracterísiticas
Las características que hemos pensado que pueden ser útiles para la clasificación de las palabras son:
- Cantidad de caracteres (Númerica)
- Proporción de consonantes por vocal (consonantes / vocales) (Numérica)
- Contiene patrones o normas ortográficas de una lengua de las que vamos a clasificar?
    + Doble uso de vocal consecutivamente como es el caso del inglés (Categórica)
    + Acentos en caso de catalán (Categórica)
    + Contiene combinaciones de consonantes (consonant clusters) propias del inglés? (Categórica)

Referencias
- <Consonant_Clusters :https://www.aprendeinglessila.com/2013/09/consonantes-ingles-clusters/>
- <Frecuencia_De_Letras_Usadas_En_Catalan: https://es.sttmedia.com/frecuencias-de-letras-catalan>
- <Frecuencia_De_Letras_Usadas_En_Catalan: https://www3.nd.edu/~busiforc/handouts/cryptography/letterfrequencies.html>

Para añadir las columnas que representen estas características, hemos aplicado las siguientes funciones a las palabras.

In [11]:
#ratio de consonantes y vocales
def ratio (word):
    vocals=0
    for c in word:
        if isVocal(c):
            vocals+=1
    return round(vocals/len(word), 4)

#isVocal?
def isVocal(c):
    if(c=='a' or c=='e' or c=='i' or c=='o' or c=='u'):
        return True
    return 
#gotAccent?
def gotAccent(word):
    #List containing all possible accentuated chars from Catalan
    accentuatedChars=[ord('à'),ord('è'),ord('é'),ord('í'),ord('ò'),ord('ó'),ord('ú')]
    for c in word:
        if ord(c) in accentuatedChars:
            return 1
    return 0
def doubleVocal(word):
    ocurrences=["aa","ee","ii","oo","uu"]
    for oc in ocurrences:
        if word.find(oc)!=-1:
            return 1
    return 0

def enCC(word):
    ocurrences=["sch","spl","shr","squ","thr","spr","scr","sph","th","tw","sw","sk","sm"]
    for oc in ocurrences:
        if word.find(oc)!=-1:
            return 1
    return 0

#Features Adding
wordsDF['ratio']=wordsDF['word'].apply(ratio)
wordsDF['cantidadLetras']=wordsDF['word'].apply(len)
wordsDF['gotAccent']=wordsDF['word'].apply(gotAccent)
wordsDF['doubleVocal']=wordsDF['word'].apply(doubleVocal)
wordsDF['enCC']=wordsDF['word'].apply(enCC)
#Reorganize DF
wordsDF=wordsDF[['word','ratio','cantidadLetras','gotAccent','doubleVocal','enCC','y']]
#Write DF to csv
wordsDF.to_csv('data/definitiveData.csv', index=False)
#Checking
print(wordsDF.to_string)

<bound method DataFrame.to_string of           word   ratio  cantidadLetras  gotAccent  doubleVocal  enCC  y
0       trobar  0.3333               6          0            0     0  0
1       direct  0.3333               6          0            0     0  1
2       parlar  0.3333               6          0            0     0  0
3        class  0.2000               5          0            0     0  1
4     position  0.5000               8          0            0     0  1
...        ...     ...             ...        ...          ...   ... ..
1971        si  0.5000               2          0            0     0  0
1972     begin  0.4000               5          0            0     0  1
1973      wife  0.5000               4          0            0     0  1
1974  caràcter  0.2500               8          1            0     0  0
1975     causa  0.6000               5          0            0     0  0

[1976 rows x 7 columns]>


### Separación del conjunto de datos en entrenamiento y test
Vamos a escoger dos tercios para el conjunto de entrenamiento y el resto para el test.
 Puntualizar que se han normalizado los datos para un mejor rendimiento del modelo. Solo se han escalado los datos numéricos, ya que sería erróneo escalar todo el dataset, con los categóricos, ya que podría dar lugar a dar menos importancia a estos últimos. 

In [12]:
#First we must separate dataframe into X and y format
X=wordsDF.iloc[:,1:6]
X_aux=wordsDF.iloc[:,1:3]
y=wordsDF.iloc[:,6]

#For better perfomance, we scale the data using an standard scaler
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(X_aux)
standardData_aux = scaler.transform(X_aux)

#Standarized DF
standardData=X

#Substitute standarized values in correspondent columns
standardData['ratio']=standardData_aux[:,0]
standardData['cantidadLetras']=standardData_aux[:,1]

#Then we separate the data frame in training and test (will be used in chosen model)
X_train, X_test, y_train, y_test = train_test_split(standardData, y, test_size=0.33, random_state=33)

## Modelos
En este problema creemos que el mejor modelo para clasificar las palabras va a ser una SVC con un kernel gaussiano (RBG) ya que tenemos un número de características bastante bajo en comparación al número de muestras. (Podríamos poner una prueba en la que probamos el kernel líneal y comprobar que nos da peores valores).

Los híperparámetros que deberemos de configurar en el modelo del SVM son:
- _C_
- _max\_iter_
- _gamma_

El resto de híperparámetros no son tan necesarios ya que no se ajustan al problema.(Explicar algún que otro parámetro y decir por qué no es útil)

In [13]:
#Model
SVM_rbf = SVC(kernel='rbf')

## Nested Cross-Validation
Para el Nested Cross-Validation haremos lo siguiente. 

In [14]:
#Hyper Grid
hyper_grid = {"C":[0.1,1,10],"max_iter":[100,1000,10000],"gamma":[.01,.1,1]}
#Inner and Outer crossvalidation, following steps from :
#https://www.analyticsvidhya.com/blog/2021/03/a-step-by-step-guide-to-nested-cross-validation/
# and
# https://inria.github.io/scikit-learn-mooc/python_scripts/cross_validation_nested.html

inner_cv=StratifiedKFold(n_splits=3, shuffle=False)
outer_cv=StratifiedKFold(n_splits=5, shuffle=False)
#GridSearch definition
search=GridSearchCV(estimator=SVM_rbf,cv=inner_cv,param_grid=hyper_grid,n_jobs=-1)
search.fit(X_train,y_train)
#Test score after nesting
test_score=cross_val_score(search,X_train,y_train,cv=outer_cv,n_jobs=-1)

#Printing results of Nested Cross-Validation
print(f"Mean score of Nested cross-validation:"f"{test_score.mean():.3f}+-{test_score.std():.3f}")
print(f"The best estimator is:"f"{search.best_estimator_}")

Mean score of Nested cross-validation:0.636+-0.029
The best estimator is:SVC(C=1, gamma=1, max_iter=1000)


## Prueba del Modelo con los parámetros encontrados

Una vez hemos encontrado los mejores híperparámetros para el SVM con Kernel Gaussiano, procedemos a entrenarlo y a realizar la predicción.

In [15]:
#Definition of SVM with best hyperparamters
SVM_rbf=SVC(kernel="rbf",C=10,gamma=0.01,max_iter=1000)
#Train Model with Train set
SVM_rbf.fit(X_train,y_train)
#Predict with X_test
y_predicted=SVM_rbf.predict(X_test)
differences=y_predicted-y_test
errors = np.count_nonzero(differences)
#Print results
print(f'Rati d\'acerts en el bloc de predicció: {(len(y_predicted)-errors)/len(y_predicted)}')

Rati d'acerts en el bloc de predicció: 0.6584992343032159
